In [1]:
import pandas as pd
import numpy as np


In [43]:
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark.sql import SparkSession


In [31]:
spark = SparkSession.builder.master("local").appName("dim_red").getOrCreate()

In [2]:
#loading data, combining all terms and counts
daily_terms = []
for mon in range(3,11):
    start = 1
    end = 32
    if mon == 3:
        start = 22
    elif mon == 4 or mon == 6 or mon == 9:
        end = 31
    elif mon == 10:
        end = 13
        
    if mon < 10:
        mon = '0'+str(mon)
    else:
        mon = str(mon)
        
        
        
        
    for day in range(start,end):
        if day < 10:
            day = '0'+str(day)
        else:
            day = str(day)
        date = '2020-'+mon+'-'+day
        df1 = pd.read_csv('./archive/dailies/'+date+'/'+date+'_top1000terms.csv', header = None)
        df1.columns = ["gram", "counts"]
        df1=df1.iloc[0:500]
        df2 = pd.read_csv('./archive/dailies/'+date+'/'+date+'_top1000bigrams.csv')
        df2=df2.iloc[0:500]
        df3 = pd.read_csv('./archive/dailies/'+date+'/'+date+'_top1000trigrams.csv')
        df3=df3.iloc[0:500]
        
        df = pd.concat([df1, df2], axis = 0)
        df = pd.concat([df,df3], axis=0)
        
        daily_terms.append(df)
        
        

In [3]:
#retrieve all terms
terms = daily_terms[0][['gram']]

for i in range(1,len(daily_terms)):
    day_t = daily_terms[i][['gram']]
    terms = pd.concat([terms,day_t], axis=0)
    terms = terms.drop_duplicates().dropna()
terms = terms.drop_duplicates().dropna()
print(len(terms))
terms.head()

66321


,gram
0,coronavirus
2,covid
3,19
4,covid19
5,people


In [4]:
terms = list(terms['gram'])

In [5]:
print(terms[0])

coronavirus


In [6]:
#create a dataframe
twitter_counts = pd.DataFrame(columns = terms)

In [7]:
twitter_counts.head()

,coronavirus,covid,19,covid19,people,trump,via,us,virus,amp,...,china qingdao,made things,enfermedad inexplicable,rid covid,city qingdao,dies cardiac,peut survivre,four days,couvre feu,confirm nevada


In [8]:
daily_terms[0].head()

,gram,counts
0,coronavirus,223843
1,NaN,86350
2,covid,67037
3,19,64625
4,covid19,56389


In [9]:
count_r = 0
for data in daily_terms:
   
    row = []
    for n in range (0,66321):
        row.append(0)

        
    twitter_counts.loc[len(twitter_counts.index)] = row
    
    #remove all nan terms    
    data = data.dropna()
     
    #populate each row with counts for each term
    for ind, r in data.iterrows():
        
        t = r['gram']
        
        if isinstance(r['counts'], int):
  
            count = r['counts']
            count = int(count)
        else:
            count = -1
        
        twitter_counts.at[count_r, t] = count
    
        
    count_r = count_r+1
        
    
            
        
       
        
        
    
    


In [10]:
#first 50 counts for terms
twitter_counts.head(50)

,coronavirus,covid,19,covid19,people,trump,via,us,virus,amp,...,china qingdao,made things,enfermedad inexplicable,rid covid,city qingdao,dies cardiac,peut survivre,four days,couvre feu,confirm nevada
0,223843,67037,64625,56389,28026,17061,16803,15701,15674,15424,...,0,0,0,0,0,0,0,0,0,0
1,245729,84031,81299,71684,29656,19862,19493,16947,15364,17404,...,0,0,0,0,0,0,0,0,0,0
2,254575,91123,87927,84368,29052,23074,21522,18568,15762,18852,...,0,0,0,0,0,0,0,0,0,0
3,253734,90652,88225,77901,27129,19520,21184,18257,14899,17315,...,0,0,0,0,0,0,0,0,0,0
4,340182,128786,124076,137568,42561,28143,33227,33261,26839,35637,...,0,0,0,0,0,0,0,0,0,0
5,347777,134682,129550,156102,42639,33096,33237,35352,29104,37370,...,0,0,0,0,0,0,0,0,0,0
6,297595,116419,116645,143953,38209,28278,29947,25339,23853,31501,...,0,0,0,0,0,0,0,0,0,0
7,301030,116988,113711,127605,41469,32519,29836,27273,23985,31752,...,0,0,0,0,0,0,0,0,0,0
8,327329,139430,133474,153681,41166,31397,33369,27433,24109,36560,...,0,0,0,0,0,0,0,0,0,0
9,309257,146153,137606,171687,40739,34108,32515,29372,25056,38653,...,0,0,0,0,0,0,0,0,0,0


In [11]:
#ensure there is no nan vallues
twitter_counts = twitter_counts.fillna(0)

In [12]:
twitter_counts.isnull().values.any()

False

In [21]:
twitter_counts.to_csv('twitter_grams_cut.csv',index=False)

In [ ]:
#since there are too many terms to use as features, I will reduce further, based on average counts
#I will eliminate all terms with average counts < 100 to focus on most popular terms

In [27]:
tc_T = twitter_counts.T

In [28]:
tc_T.head()

,0,1,2,3,4,5,6,7,8,9,...,195,196,197,198,199,200,201,202,203,204
coronavirus,223843,245729,254575,253734,340182,347777,297595,301030,327329,309257,...,136538,128024,152965,161959,159071,131745,129660,107782,112491,131308
covid,67037,84031,91123,90652,128786,134682,116419,116988,139430,146153,...,183192,170023,223703,238661,202421,176613,171228,133576,125547,171556
19,64625,81299,87927,88225,124076,129550,116645,113711,133474,137606,...,159520,148131,192462,209004,179382,157906,152629,117935,110501,152225
covid19,56389,71684,84368,77901,137568,156102,143953,127605,153681,171687,...,129828,121797,175416,144785,132536,120313,119745,88995,83437,138860
people,28026,29656,29052,27129,42561,42639,38209,41469,41166,40739,...,35594,32430,42042,45219,35731,29229,28010,24509,21972,28401


In [32]:
spark_tc = spark.createDataFrame(tc_T) #since there are a lot of terms, I will use pyspark

In [46]:
spark_tc.show(5)

+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-----

In [49]:
from operator import add
from functools import reduce

import pyspark.sql.functions as F

In [53]:



numeric_col_list = spark_tc.columns

print(numeric_col_list)


['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '15

In [55]:
#sums all the values in each row 
#(you may ignore the error, it occured due to running this command twice)
tot = spark_tc.withColumn('total',F.sum([F.col(x) for x in numeric_col_list])

SyntaxError: unexpected EOF while parsing (Temp/ipykernel_26948/845105252.py, line 1)

In [56]:
tot.show(5)

+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-----

In [59]:
tot = tot.withColumn('avr', F.col('total')/205).select(F.col('avr'))#compute average count

In [60]:
tot.show(5)

+------------------+
|               avr|
+------------------+
|219748.01463414636|
| 186334.6487804878|
|173481.25853658537|
| 149806.7024390244|
|           35661.2|
+------------------+
only showing top 5 rows



In [62]:
#extract the terms list
terms_list = twitter_counts.columns
print(terms_list)

Index(['coronavirus', 'covid', '19', 'covid19', 'people', 'trump', 'via', 'us',
       'virus', 'amp',
       ...
       'china qingdao', 'made things', 'enfermedad inexplicable', 'rid covid',
       'city qingdao', 'dies cardiac', 'peut survivre', 'four days',
       'couvre feu', 'confirm nevada'],
      dtype='object', length=66321)


In [70]:
from pyspark.sql.types import StringType


#create a pyspark df from the terms
cols_spark = spark.createDataFrame(terms_list, StringType())

In [71]:
cols_spark.show(5)

+-----------+
|      value|
+-----------+
|coronavirus|
|      covid|
|         19|
|    covid19|
|     people|
+-----------+
only showing top 5 rows



In [81]:
from pyspark.sql.functions import monotonically_increasing_id, row_number
from pyspark.sql import Window
#add an index column to the terms df
cols_spark =  cols_spark.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))

In [83]:
#add an index column to the average count df
tot =  tot.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))

In [84]:
#join the two columns and remove the index
final_df = tot.join(cols_spark, tot.row_idx == cols_spark.row_idx).drop("row_idx")

In [85]:
final_df.show(5)

+------------------+-----------+
|               avr|      value|
+------------------+-----------+
|219748.01463414636|coronavirus|
| 186334.6487804878|      covid|
|173481.25853658537|         19|
| 149806.7024390244|    covid19|
|           35661.2|     people|
+------------------+-----------+
only showing top 5 rows



In [86]:
#only select terms which have average count > 100
reduced_terms = final_df.select('value').filter(final_df.avr > 100)

In [87]:
reduced_terms.show(5)

+-----------+
|      value|
+-----------+
|coronavirus|
|      covid|
|         19|
|    covid19|
|     people|
+-----------+
only showing top 5 rows



In [88]:
#new number of terms
reduced_terms.count()

1874

In [91]:
twit_l = list(reduced_terms.toPandas()['value'])

In [92]:
twit = twitter_counts[twit_l]

In [94]:
twit.shape

(205, 1874)

In [95]:
#First attept at feature reduction from above ^, took too much computational time
#rem_cnt = 0
#cnt = 0
#twit = twitter_counts
#cols = twit.columns
#depricate columns where avrage count < 100
#for term in cols:
#    avr = twit.loc[:, term].mean()
#    cnt = cnt+1
#    print(cnt)
#    if avr < 100.0:
#        twit = twit.drop(term, axis = 1)
#        rem_cnt=rem_cnt+1
#        print(twit.shape)
        

In [96]:
twit.to_csv('twitter_grams_cut2.csv',index=False)

In [ ]:
twitter_counts.head(10)

In [36]:
twitter_counts.dtypes

coronavirus                                          int64
covid                                                int64
19                                                   int64
covid19                                              int64
people                                               int64
                                                     ...  
28 dias                                              int64
reconfinement covid19                                int64
news outlets                                         int64
12 oct                                               int64
puebloenresistenciaantiimperialista fabiozavarsep    int64
Length: 137780, dtype: object

In [37]:
twitter_counts.astype({'covid 19': 'int64'}).dtypes

coronavirus                                          int64
covid                                                int64
19                                                   int64
covid19                                              int64
people                                               int64
                                                     ...  
28 dias                                              int64
reconfinement covid19                                int64
news outlets                                         int64
12 oct                                               int64
puebloenresistenciaantiimperialista fabiozavarsep    int64
Length: 137780, dtype: object

In [40]:
twitter_counts[["covid 19"]].dtypes

covid 19    object
dtype: object

In [41]:
twitter_counts.to_csv('twitter_grams.csv',index=False)

In [ ]:
#manually placed to hdfs

In [ ]:
#Generate a Json string for schema.

In [42]:
t_cols = twitter_counts.columns.values.tolist()


In [46]:
with open('json_schema.txt', 'w', encoding="utf-8") as f:
    f.write('{\'fields\': [')
    for name in t_cols:
        f.write('{\'metadata\': {}, \'name\': \''+name+'\', \'nullable\': False, \'type\': \'long\'},')
        f.write('\n')
    f.write('],\n\'type\': \'struct\'})')

In [ ]:
#Now, processing the hospitalization data.

In [89]:
hospital_data = pd.read_csv('./covid-hospitalizations.csv')

In [90]:
hospital_data.head()

,entity,iso_code,date,indicator,value
0,Algeria,DZA,2020-07-17,Daily ICU occupancy,62.000
1,Algeria,DZA,2020-07-17,Daily ICU occupancy per million,1.381
2,Algeria,DZA,2020-07-18,Daily ICU occupancy,67.000
3,Algeria,DZA,2020-07-18,Daily ICU occupancy per million,1.492
4,Algeria,DZA,2020-07-20,Daily ICU occupancy,64.000


In [91]:
hospital_data.dtypes

entity        object
iso_code      object
date          object
indicator     object
value        float64
dtype: object

In [92]:
hospital_data =hospital_data.loc[hospital_data['date'] > '2020-03-21'] #set start date

In [93]:
hospital_data = hospital_data.loc[hospital_data['date'] < '2020-10-13'] #set end date

In [94]:
hosp_world_average = hospital_data.groupby(['indicator', 'date'], as_index=False).mean() #average the data

In [95]:
len(hosp_world_average)

1640

In [96]:
#hospital_data =hospital_data.loc[hospital_data['indicator'] == 'Daily ICU occupancy']

In [97]:
hosp_world_average['indicator'].unique()

array(['Daily ICU occupancy', 'Daily ICU occupancy per million',
       'Daily hospital occupancy', 'Daily hospital occupancy per million',
       'Weekly new ICU admissions',
       'Weekly new ICU admissions per million',
       'Weekly new hospital admissions',
       'Weekly new hospital admissions per million'], dtype=object)

In [98]:
#list the columns for targets
hosp_col_list = hosp_world_average['indicator'].unique()

In [99]:
#create a dataframe with each hospitalization unit indicator
def create_df(df, hosp_col_list_):
    list_ = df['date'].unique()
    df_new = pd.DataFrame(list_, columns=['date'])
    for item in hosp_col_list_:
        df_c = df.loc[df['indicator'] == item]
        df_c= df_c.drop('indicator', axis = 1)
        df_c = df_c.drop('date', axis = 1)
        df_c = df_c.rename({'value': item}, axis=1)
        df_new = pd.concat([df_new.reset_index(drop=True),df_c.reset_index(drop=True)], axis=1)
        
        
        
    return df_new

In [100]:
avr_hosp_data = create_df(hosp_world_average, hosp_col_list)

In [101]:
avr_hosp_data.tail()

,date,Daily ICU occupancy,Daily ICU occupancy per million,Daily hospital occupancy,Daily hospital occupancy per million,Weekly new ICU admissions,Weekly new ICU admissions per million,Weekly new hospital admissions,Weekly new hospital admissions per million
200,2020-10-08,685.103448,13.741103,2710.678571,71.677786,456.200000,16.509400,3557.705882,80.726235
201,2020-10-09,671.566667,13.529033,2644.896552,71.012552,466.800000,17.060400,3669.000000,83.427412
202,2020-10-10,702.586207,14.305862,2872.222222,77.871000,463.600000,16.900800,3663.000000,83.082278
203,2020-10-11,687.933333,14.296467,2784.241379,81.458138,174.142857,9.175429,2330.551724,69.283000
204,2020-10-12,705.333333,14.502833,2901.103448,81.029690,471.200000,17.392400,3962.705882,92.175824


In [102]:
len(avr_hosp_data)

205

In [103]:
avr_hosp_data.to_csv('avr_hosp_data.csv',index=False) #manually uploaded to hdfs

In [ ]:
#Repeat above but for the US only

In [104]:
hospital_data_USA = hospital_data.loc[hospital_data['entity'] == 'United States']

In [106]:
hospital_data_USA=hospital_data_USA.drop(['entity', 'iso_code'], axis =1)

In [107]:
us_ind = hospital_data_USA['indicator'].unique()

In [108]:
hosp_data_USA = create_df(hospital_data_USA, us_ind)

In [109]:
hosp_data_USA  = hosp_data_USA.fillna(-1)

In [110]:
hosp_data_USA.head(200)

,date,Daily ICU occupancy,Daily ICU occupancy per million,Daily hospital occupancy,Daily hospital occupancy per million,Weekly new hospital admissions,Weekly new hospital admissions per million
0,2020-07-15,9245.0,27.329,33760.0,99.796,30552.0,90.313
1,2020-07-16,9797.0,28.960,38539.0,113.923,31352.0,92.678
2,2020-07-17,10700.0,31.630,40062.0,118.425,31679.0,93.645
3,2020-07-18,10963.0,32.407,40376.0,119.353,31870.0,94.209
4,2020-07-19,10267.0,30.350,37897.0,112.025,32804.0,96.970
...,...,...,...,...,...,...,...
85,2020-10-08,8624.0,25.493,31015.0,91.682,-1.0,-1.000
86,2020-10-09,8723.0,25.786,31207.0,92.249,-1.0,-1.000
87,2020-10-10,8731.0,25.809,30981.0,91.581,-1.0,-1.000
88,2020-10-11,8819.0,26.069,31435.0,92.923,-1.0,-1.000


In [114]:
hosp_data_USA.to_csv('hosp_data_USA.csv',index=False)

In [116]:
#repeat the above for England
hospital_data_Eng = hospital_data.loc[hospital_data['entity'] == 'England']
hospital_data_Eng=hospital_data_Eng.drop(['entity', 'iso_code'], axis =1)
uk_ind = hospital_data_Eng['indicator'].unique()
hosp_data_Eng = create_df(hospital_data_Eng, uk_ind)
hosp_data_Eng  = hosp_data_Eng.fillna(-1)
hosp_data_Eng.head(200)


,date,Daily hospital occupancy,Daily hospital occupancy per million,Weekly new hospital admissions,Weekly new hospital admissions per million,Daily ICU occupancy,Daily ICU occupancy per million
0,2020-03-22,2670.0,47.215,7340.0,129.797,1494.0,26.419
1,2020-03-23,3183.0,56.286,8393.0,148.417,1788.0,31.618
2,2020-03-24,3598.0,63.625,9633.0,170.345,1950.0,34.483
3,2020-03-25,4402.0,77.843,10729.0,189.726,2097.0,37.082
4,2020-03-26,5226.0,92.414,12056.0,213.192,2264.0,40.035
...,...,...,...,...,...,...,...
195,2020-10-03,2290.0,40.495,2927.0,51.760,-1.0,-1.000
196,2020-10-04,2435.0,43.059,3090.0,54.642,-1.0,-1.000
197,2020-10-05,2705.0,47.834,3235.0,57.206,-1.0,-1.000
198,2020-10-06,2903.0,51.335,3408.0,60.265,-1.0,-1.000


In [117]:
hosp_data_Eng.to_csv('hosp_data_Eng.csv',index=False)

In [118]:
#repeat the above for Germany
hospital_data_Ger = hospital_data.loc[hospital_data['entity'] == 'Germany']
hospital_data_Ger=hospital_data_Ger.drop(['entity', 'iso_code'], axis =1)
ger_ind = hospital_data_Ger['indicator'].unique()
hosp_data_Ger = create_df(hospital_data_Ger, ger_ind)
hosp_data_Ger  = hosp_data_Ger.fillna(-1)
hosp_data_Ger.head(200)


,date,Daily ICU occupancy,Daily ICU occupancy per million,Weekly new hospital admissions,Weekly new hospital admissions per million
0,2020-03-22,364.0,4.366,2020.0,24.229
1,2020-03-23,451.0,5.410,2236.0,26.820
2,2020-03-24,616.0,7.389,2538.0,30.443
3,2020-03-25,789.0,9.464,3003.0,36.020
4,2020-03-26,824.0,9.884,3494.0,41.910
...,...,...,...,...,...
195,2020-10-03,403.0,4.834,848.0,10.172
196,2020-10-04,424.0,5.086,881.0,10.567
197,2020-10-05,446.0,5.350,891.0,10.687
198,2020-10-06,449.0,5.386,960.0,11.515


In [119]:
hosp_data_Ger.to_csv('hosp_data_Ger.csv',index=False)

In [120]:
#repeat the above for Australia
hospital_data_Aus = hospital_data.loc[hospital_data['entity'] == 'Australia']
hospital_data_Aus=hospital_data_Aus.drop(['entity', 'iso_code'], axis =1)
Aus_ind = hospital_data_Aus['indicator'].unique()
hosp_data_Aus = create_df(hospital_data_Aus, Aus_ind)
hosp_data_Aus  = hosp_data_Aus.fillna(-1)
hosp_data_Aus.head(200)



,date,Daily ICU occupancy,Daily ICU occupancy per million,Daily hospital occupancy,Daily hospital occupancy per million
0,2020-03-31,82.0,3.132,426.0,16.274
1,2020-04-01,86.0,3.285,441.0,16.847
2,2020-04-02,92.0,3.514,415.0,15.853
3,2020-04-03,93.0,3.553,457.0,17.458
4,2020-04-04,95.0,3.629,490.0,18.718
...,...,...,...,...,...
191,2020-10-08,4.0,0.153,28.0,1.070
192,2020-10-09,3.0,0.115,31.0,1.184
193,2020-10-10,1.0,0.038,31.0,1.184
194,2020-10-11,1.0,0.038,32.0,1.222


In [121]:
hosp_data_Aus.to_csv('hosp_data_Aus.csv',index=False)

In [122]:
#Repeat the above for Chile
hospital_data_Chil = hospital_data.loc[hospital_data['entity'] == 'Chile']
hospital_data_Chil=hospital_data_Chil.drop(['entity', 'iso_code'], axis =1)
Chil_ind = hospital_data_Chil['indicator'].unique()
hosp_data_Chil = create_df(hospital_data_Chil, Chil_ind)
hosp_data_Chil  = hosp_data_Chil.fillna(-1)
hosp_data_Chil.head(200)


,date,Daily ICU occupancy,Daily ICU occupancy per million,Weekly new hospital admissions,Weekly new hospital admissions per million,Weekly new ICU admissions,Weekly new ICU admissions per million
0,2020-04-01,173.0,8.825,660.0,33.667,195.0,9.947
1,2020-04-02,200.0,10.202,644.0,32.851,194.0,9.896
2,2020-04-03,237.0,12.090,636.0,32.443,183.0,9.335
3,2020-04-04,280.0,14.283,603.0,30.759,168.0,8.570
4,2020-04-05,307.0,15.660,580.0,29.586,180.0,9.182
...,...,...,...,...,...,...,...
190,2020-10-08,812.0,41.421,-1.0,-1.000,-1.0,-1.000
191,2020-10-09,821.0,41.880,-1.0,-1.000,-1.0,-1.000
192,2020-10-10,817.0,41.676,-1.0,-1.000,-1.0,-1.000
193,2020-10-11,791.0,40.349,-1.0,-1.000,-1.0,-1.000


In [123]:
hosp_data_Chil.to_csv('hosp_data_Chil.csv',index=False)

In [124]:
#repeat the above for South Africa
hospital_data_SA = hospital_data.loc[hospital_data['entity'] == 'South Africa']
hospital_data_SA=hospital_data_SA.drop(['entity', 'iso_code'], axis =1)
SA_ind = hospital_data_SA['indicator'].unique()
hosp_data_SA = create_df(hospital_data_SA, SA_ind)
hosp_data_SA  = hosp_data_SA.fillna(-1)
hosp_data_SA.head(200)


,date,Daily ICU occupancy,Daily ICU occupancy per million,Daily hospital occupancy,Daily hospital occupancy per million,Weekly new hospital admissions,Weekly new hospital admissions per million
0,2020-03-22,0.0,0.000,96.0,1.603,269.0,4.491
1,2020-03-23,0.0,0.000,101.0,1.686,390.0,6.512
2,2020-03-24,0.0,0.000,184.0,3.072,462.0,7.714
3,2020-03-25,2.0,0.033,183.0,3.055,1081.0,18.049
4,2020-03-26,2.0,0.033,185.0,3.089,1426.0,23.809
...,...,...,...,...,...,...,...
195,2020-10-03,600.0,10.018,-1.0,-1.000,-1.0,-1.000
196,2020-10-04,600.0,10.018,-1.0,-1.000,-1.0,-1.000
197,2020-10-05,600.0,10.018,-1.0,-1.000,-1.0,-1.000
198,2020-10-06,600.0,10.018,-1.0,-1.000,-1.0,-1.000


In [125]:
hosp_data_SA.to_csv('hosp_data_SA.csv',index=False)

In [126]:
#Repeat the above for Israel
hospital_data_Isr = hospital_data.loc[hospital_data['entity'] == 'Israel']
hospital_data_Isr=hospital_data_Isr.drop(['entity', 'iso_code'], axis =1)
Isr_ind = hospital_data_Isr['indicator'].unique()
hosp_data_Isr = create_df(hospital_data_Isr, Isr_ind)
hosp_data_Isr  = hosp_data_Isr.fillna(-1)
hosp_data_Isr.head(200)


,date,Daily hospital occupancy,Daily hospital occupancy per million,Weekly new ICU admissions,Weekly new ICU admissions per million,Weekly new hospital admissions,Weekly new hospital admissions per million,Daily ICU occupancy,Daily ICU occupancy per million
0,2020-03-22,344.0,36.406,16.0,1.693,273.0,28.892,125.0,13.229
1,2020-03-23,380.0,40.216,19.0,2.011,307.0,32.490,125.0,13.229
2,2020-03-24,424.0,44.872,21.0,2.222,340.0,35.983,118.0,12.488
3,2020-03-25,443.0,46.883,24.0,2.540,368.0,38.946,123.0,13.017
4,2020-03-26,501.0,53.021,25.0,2.646,419.0,44.343,133.0,14.076
...,...,...,...,...,...,...,...,...,...
195,2020-10-03,1835.0,194.200,127.0,13.441,1687.0,178.537,-1.0,-1.000
196,2020-10-04,1965.0,207.959,121.0,12.806,1669.0,176.632,-1.0,-1.000
197,2020-10-05,1928.0,204.043,125.0,13.229,1708.0,180.760,-1.0,-1.000
198,2020-10-06,1925.0,203.725,126.0,13.335,1644.0,173.987,-1.0,-1.000


In [127]:
hosp_data_Isr.to_csv('hosp_data_Isr.csv',index=False)

In [128]:
#repeat the above for Japan
hospital_data_Jap = hospital_data.loc[hospital_data['entity'] == 'Japan']
hospital_data_Jap=hospital_data_Jap.drop(['entity', 'iso_code'], axis =1)
Jap_ind = hospital_data_Jap['indicator'].unique()
hosp_data_Jap = create_df(hospital_data_Jap, Jap_ind)
hosp_data_Jap  = hosp_data_Jap.fillna(-1)
hosp_data_Jap.head(200)


,date,Daily hospital occupancy,Daily hospital occupancy per million,Daily ICU occupancy,Daily ICU occupancy per million
0,2020-04-28,5514.0,44.485,341.0,2.751
1,2020-05-07,4436.0,35.788,251.0,2.025
2,2020-05-13,3423.0,27.616,184.0,1.484
3,2020-05-21,2058.0,16.603,143.0,1.154
4,2020-05-27,1369.0,11.045,101.0,0.815
5,2020-06-03,1015.0,8.189,88.0,0.710
6,2020-06-10,781.0,6.301,70.0,0.565
7,2020-06-17,587.0,4.736,58.0,0.468
8,2020-06-24,559.0,4.510,40.0,0.323
9,2020-07-01,696.0,5.615,36.0,0.290


In [129]:
hosp_data_Jap.to_csv('hosp_data_Jap.csv',index=False)